In [1]:
import pandas as pd
import re
import numpy as np
pd.set_option('display.max_columns', 100)  # or 1000
pd.set_option('display.max_rows', 1000)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199
from soynlp.hangle import levenshtein, jamo_levenshtein

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [2]:
lexicon = pd.read_csv('OCR_lexicon.csv', header=None, names=['가공식품'])

In [173]:
# lexicon.to_csv('OCR_lexicon_pre.csv', index=False)

In [ ]:
lexicon[lexicon['가공식품'].apply(lambda x : bool(re.compile(']').search(x)))]

In [4]:
lexicon['preprocess'] = lexicon['가공식품'].apply(lambda x : x.replace(',', '_' ))
lexicon['preprocess'] = lexicon['preprocess'].apply(lambda x : x.replace(' ', '_' ))

In [11]:
lexicon['preprocess'] = lexicon['preprocess'].str.replace('(', '_')
lexicon['preprocess'] = lexicon['preprocess'].str.replace(')', '_')
lexicon['preprocess'] = lexicon['preprocess'].str.replace('[', '_')
lexicon['preprocess'] = lexicon['preprocess'].str.replace(']', '_')
lexicon['preprocess'] = lexicon['preprocess'].str.replace('__', '_')

In [6]:
# 마지막에 _ 만 달랑 있는애들 _ 제거하기
lexicon['preprocess'] = lexicon['preprocess'].apply(lambda x : x[:-1] if bool(re.compile('.*_$').search(x)) else x )
# 앞에 _ 만 있는애들 _ 제거하기
lexicon['preprocess'] = lexicon['preprocess'].apply(lambda x : x[1:] if bool(re.compile('^_.*').search(x)) else x )

## SAVE POINT

In [5]:
lexicon = pd.read_csv('OCR_lexicon_pre.csv')
lexicon

,가공식품,preprocess
0,"비스켓, 패스트푸드",비스켓_패스트푸드
1,"비스켓 ,KFC",비스켓_KFC
2,"엔칠라다, 패스트푸드",엔칠라다_패스트푸드
3,애플파이_맥도날드,애플파이_맥도날드
4,애플파이_버거킹,애플파이_버거킹
...,...,...
2450,에그 베이글 스타벅스,에그_베이글_스타벅스
2451,베이컨 치즈베이글 스타벅스,베이컨_치즈베이글_스타벅스
2452,포크 커틀릿 샌드위치 스타벅스,포크_커틀릿_샌드위치_스타벅스
2453,만한대찬 우육면 매운 소고기맛 Uni President,만한대찬_우육면_매운_소고기맛_Uni_President


In [6]:
example = ['만한대장','매은','소고기맛'] 

In [19]:
ed_list = []
for word in lexicon['preprocess'].loc[[2453]]:
    word_ed = []
    word_split = word.split('_')
    for word_ in word_split:
        best_of_word = []
        for each in example :
            ed = levenshtein(each, word_)
            er = ed/(max(len(word_), len(each)))
            if er > 0.5:
                er= 1
            best_of_word.append(er)
        word_ed.append(min(best_of_word))
    print(f'word_ed : {word_ed}')
    n_1 = word_ed.count(1)
    if n_1 >1:
        for _ in range(n_1 -1):
            word_ed.remove(1)
        print('word_ed after pop : ', word_ed)
        div_val = len(word_split) - n_1 + 1
        print(f'div val : {div_val}')
        ed_list.append(sum(word_ed)/div_val)
    else:
        ed_list.append(sum(word_ed)/len(word_split))
#     ed_list.append(word_ed/len(word_split))

# lexicon['ed'] = ed_list

word_ed : [0.25, 1, 0.5, 0.0, 1, 1]
word_ed after pop :  [0.25, 0.5, 0.0, 1]
div val : 4


In [17]:
why = lexicon[lexicon['ed'] == lexicon['ed'].min()]
why

,가공식품,preprocess,ed
752,소고기육포,소고기육포,0.4


In [18]:
lexicon.tail(100)

,가공식품,preprocess,ed
2355,통새우 전복죽 풀무원,통새우_전복죽_풀무원,1.000000
2356,새알 동지팥죽 풀무원,새알_동지팥죽_풀무원,1.000000
2357,쇠고기버섯죽 풀무원,쇠고기버섯죽_풀무원,1.000000
2358,오곡 삼계죽 풀무원,오곡_삼계죽_풀무원,1.000000
2359,모락모락 고기잡채 볶음밥 풀무원,모락모락_고기잡채_볶음밥_풀무원,1.000000
2360,"곤드레나물솥밥(1인분, 용기형) 풀무원",곤드레나물솥밥_1인분_용기형_풀무원,1.000000
2361,잇슬림 150 플레인프로틴 풀무원,잇슬림_150_플레인프로틴_풀무원,1.000000
2362,잇슬림 150 토마토치킨프로틴 풀무원,잇슬림_150_토마토치킨프로틴_풀무원,1.000000
2363,잇슬림 150 커리치킨프로틴 풀무원,잇슬림_150_커리치킨프로틴_풀무원,1.000000
2364,잇슬림 150 캐비지치킨프로틴 풀무원,잇슬림_150_캐비지치킨프로틴_풀무원,1.000000


In [37]:
ed_list = []
for word in why['preprocess']:
    word_ed = []
    word_split = word.split('_')
    print(word)
    for word_ in word_split:
        best_of_word = []
        for each in example :
            ed = levenshtein(each, word_)
            er = ed/(max(len(word_), len(each)))
            if er > 0.5:
                er= 1
            print(f'{word_} -> {each} : ED {ed}, ER {er}')
            best_of_word.append(er)
        print(f'{word_} -> {example} : best_of_word : {best_of_word}')
        word_ed.append(min(best_of_word))
    print('word_ed : ', word_ed)
    n_1 = word_ed.count(1)
    if n_1 >1:
        for _ in range(n_1 -1):
            word_ed.pop(1)
            print(f'after pop WORD_ED  {word_ed}')
            div_val = len(word_split) - n_1 + 1
            ed_list.append(sum(word_ed)/len(div_val))
    else:
        ed_list.append(sum(word_ed)/len(word_split))

소고기육포
소고기육포 -> 만한대장 : ED 5, ER 1
소고기육포 -> 매은 : ED 5, ER 1
소고기육포 -> 소고기맛 : ED 2, ER 0.4
소고기육포 -> ['만한대장', '매은', '소고기맛'] : best_of_word : [1, 1, 0.4]
word_ed :  [0.4]


In [33]:
for _ in range(0 -1):
    print('몇번')

In [248]:
why = lexicon.loc[[2412]]

In [252]:
ed_list = []
for word in why['preprocess']:
    word_ed = 0
    word_split = word.split('_')
    print(word)
    for word_ in word_split:
        each_best = []    
        for each in example :
            ed = levenshtein(each, word_)
            er = ed/(max(len(word_), len(each)))
            if er > 0.5:
                er= 1
            print(f'{word_} -> {each} : ED {ed}, ER {er}')
            each_best.append(er)
        print(f'{word_} -> {example} : each_list : {each_best}')
        print(min(each_best))
        word_ed += min(each_best)
    ed_list.append(word_ed/len(word_split))

노니&깔라만시_풀무원
노니&깔라만시 -> 오사카식무종 : ED 7, ER 0.5
노니&깔라만시 -> 풀무언 : ED 7, ER 0.5
노니&깔라만시 -> ['오사카식무종', '풀무언'] : each_list : [0.5, 0.5]
0.5
풀무원 -> 오사카식무종 : ED 5, ER 0.5
풀무원 -> 풀무언 : ED 1, ER 0.3333333333333333
풀무원 -> ['오사카식무종', '풀무언'] : each_list : [0.5, 0.3333333333333333]
0.3333333333333333
케일&셀러리_풀무원
케일&셀러리 -> 오사카식무종 : ED 6, ER 0.5
케일&셀러리 -> 풀무언 : ED 6, ER 0.5
케일&셀러리 -> ['오사카식무종', '풀무언'] : each_list : [0.5, 0.5]
0.5
풀무원 -> 오사카식무종 : ED 5, ER 0.5
풀무원 -> 풀무언 : ED 1, ER 0.3333333333333333
풀무원 -> ['오사카식무종', '풀무언'] : each_list : [0.5, 0.3333333333333333]
0.3333333333333333
파래김자반_풀무원
파래김자반 -> 오사카식무종 : ED 6, ER 0.5
파래김자반 -> 풀무언 : ED 5, ER 0.5
파래김자반 -> ['오사카식무종', '풀무언'] : each_list : [0.5, 0.5]
0.5
풀무원 -> 오사카식무종 : ED 5, ER 0.5
풀무원 -> 풀무언 : ED 1, ER 0.3333333333333333
풀무원 -> ['오사카식무종', '풀무언'] : each_list : [0.5, 0.3333333333333333]
0.3333333333333333
국산콩두부단단한부침용380g_풀무원
국산콩두부단단한부침용380g -> 오사카식무종 : ED 15, ER 0.5
국산콩두부단단한부침용380g -> 풀무언 : ED 15, ER 0.5
국산콩두부단단한부침용380g -> ['오사카식무종', '풀무언'] : each_list :

In [154]:
ed_list

[0.42857142857142855, 0.42857142857142855]

---------------

# 다합치기

In [5]:
lexicon = pd.read_csv('OCR_lexicon_pre.csv')
lexicon

,가공식품,preprocess
0,"비스켓, 패스트푸드",비스켓_패스트푸드
1,"비스켓 ,KFC",비스켓_KFC
2,"엔칠라다, 패스트푸드",엔칠라다_패스트푸드
3,애플파이_맥도날드,애플파이_맥도날드
4,애플파이_버거킹,애플파이_버거킹
...,...,...
2450,에그 베이글 스타벅스,에그_베이글_스타벅스
2451,베이컨 치즈베이글 스타벅스,베이컨_치즈베이글_스타벅스
2452,포크 커틀릿 샌드위치 스타벅스,포크_커틀릿_샌드위치_스타벅스
2453,만한대찬 우육면 매운 소고기맛 Uni President,만한대찬_우육면_매운_소고기맛_Uni_President


In [16]:
lexicon['join'] = lexicon['preprocess'].str.replace('_', '')
lexicon

,가공식품,preprocess,join,ed
0,"비스켓, 패스트푸드",비스켓_패스트푸드,비스켓패스트푸드,1.250000
1,"비스켓 ,KFC",비스켓_KFC,비스켓KFC,1.666667
2,"엔칠라다, 패스트푸드",엔칠라다_패스트푸드,엔칠라다패스트푸드,1.111111
3,애플파이_맥도날드,애플파이_맥도날드,애플파이맥도날드,1.250000
4,애플파이_버거킹,애플파이_버거킹,애플파이버거킹,1.428571
...,...,...,...,...
2450,에그 베이글 스타벅스,에그_베이글_스타벅스,에그베이글스타벅스,1.111111
2451,베이컨 치즈베이글 스타벅스,베이컨_치즈베이글_스타벅스,베이컨치즈베이글스타벅스,1.000000
2452,포크 커틀릿 샌드위치 스타벅스,포크_커틀릿_샌드위치_스타벅스,포크커틀릿샌드위치스타벅스,1.000000
2453,만한대찬 우육면 매운 소고기맛 Uni President,만한대찬_우육면_매운_소고기맛_Uni_President,만한대찬우육면매운소고기맛UniPresident,0.720000


In [17]:
example = '소고기맛매운만한대찬'

In [18]:
ed_list = []
for word  in lexicon['join']:
    ed = levenshtein(example, word)
    er = ed/(len(word))
    ed_list.append(er)
lexicon['ed'] = ed_list

In [19]:
lexicon[lexicon['ed']== lexicon['ed'].min()]

,가공식품,preprocess,join,ed
763,"불고기스모크햄, 한성기업",불고기스모크햄_한성기업,불고기스모크햄한성기업,0.727273
1388,오뚜기카레(매운맛)_오뚜기,오뚜기카레_매운맛_오뚜기,오뚜기카레매운맛오뚜기,0.727273
2350,소고기버섯비빔밥 풀무원,소고기버섯비빔밥_풀무원,소고기버섯비빔밥풀무원,0.727273


##### 다합치기 : 순서가 바뀔경우 실패한다